<a href="https://colab.research.google.com/github/Manvit07/FRIDAY_FOSS/blob/main/Friday_code_fixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00


In [29]:
!pip install selenium webdriver-manager


In [2]:
!huggingface-cli logout

Not logged in!


In [3]:
!pip install pandas nltk transformers torch tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!pip install transformers datasets torch pandas nltk tqdm scikit-learn
!pip install sentencepiece  # Required for CodeT5 tokenizer

In [5]:
!git clone https://github.com/salesforce/CodeT5.git
%cd CodeT5/CodeT5+

Cloning into 'CodeT5'...
remote: Enumerating objects: 295, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 295 (delta 105), reused 65 (delta 65), pack-reused 155 (from 1)
Receiving objects: 100% (295/295), 10.91 MiB | 7.96 MiB/s, done.
Resolving deltas: 100% (151/151), done.
/content/CodeT5/CodeT5+


In [6]:

!python run_gen.py \
    --model_name_or_path Salesforce/codet5p-base \
    --do_train \
    --do_eval \
    --train_file ../dataset.csv \
    --output_dir ./codet5p-fine-tuned \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --num_train_epochs 10 \
    --learning_rate 5e-5 \
    --save_strategy epoch \
    --evaluation_strategy epoch \
    --logging_steps 10 \
    --save_total_limit 2 \
    --fp16

python3: can't open file '/content/CodeT5/CodeT5+/run_gen.py': [Errno 2] No such file or directory


In [30]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import T5ForConditionalGeneration,T5Tokenizer,Trainer,TrainingArguments,AutoModelForSeq2SeqLM, AutoTokenizer
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

import os


In [8]:
import json

In [10]:

dataset = load_dataset("json", data_files="/content/dataset.json")


print(type(dataset))


print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['_key', 'trigger_function', 'explanation', 'corrected_code'],
        num_rows: 44
    })
})


In [12]:
dataset = load_dataset("json", data_files="/content/dataset.json")

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
model_name = "Salesforce/codet5p-220m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5p-220m").to("cuda")

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/446M [00:00<?, ?B/s]

In [18]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples["trigger_function"], padding="max_length", truncation=True, max_length=128)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["corrected_code"], padding="max_length", truncation=True, max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [19]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5p-220m").to("cuda")

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [21]:
training_args = TrainingArguments(
    output_dir="./codet5p-fine-tuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=16,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["train"],
)



trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,6.603034
2,7.657400,6.003687
3,7.657400,5.616373
4,5.974900,5.172060
5,5.357800,4.849308
6,5.357800,4.625834
7,4.930600,4.509279
8,4.930600,4.429254
9,4.597100,4.384026
10,4.617700,4.336769


TrainOutput(global_step=96, training_loss=5.09811011950175, metrics={'train_runtime': 448.7033, 'train_samples_per_second': 1.569, 'train_steps_per_second': 0.214, 'total_flos': 107176588738560.0, 'train_loss': 5.09811011950175, 'epoch': 16.0})

In [32]:
def search_stackoverflow(error_text):

    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  #
    driver = webdriver.Chrome(service=service, options=options)


    search_query = f"site:stackoverflow.com {error_text}"
    driver.get("https://www.google.com")

    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)

    time.sleep(2)


    results = driver.find_elements(By.XPATH, "//div[@class='tF2Cxc']//a")
    links = [result.get_attribute("href") for result in results[:3]]

    driver.quit()
    return links if links else ["No solutions found on Stack Overflow"]

In [27]:
def fix_code(error_text):
    model.eval()


    inputs = tokenizer(
        "Fix this code:\n" + error_text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    ).to("cuda")

    with torch.no_grad():
        output_ids = model.generate(inputs["input_ids"], max_length=512, num_beams=5)

    fixed_code = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return fixed_code if fixed_code.strip() else "⚠️ No meaningful suggestion generated!"


error_code = """
if x = 5:
    print("x is 5")
"""

fix_suggestion = fix_code(error_code)
print(f"💡 Fixed Code:\n{fix_suggestion}")


💡 Fixed Code:

if x == 5:
    print("x is 5")
